# ANOVA

Initial commands

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf') #setting figure format to vector when exported
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['text.usetex'] = True
from scipy.stats import f
from scipy.stats import f_oneway
from scipy.special import binom
from scipy.stats import ttest_ind
from itertools import combinations 

Reading data

In [2]:
data=pd.DataFrame({'5%':[48,55,103,76,62,69],'10%':[83,117,90,123,133,103],'15%':[97,124,128,119,110,np.nan],'20%':[131,172,152,158,126,138]},columns=['5%','10%','15%','20%'])
display(data)
alpha=0.05 #significance level

,5%,10%,15%,20%
0,48,83,97.0,131
1,55,117,124.0,172
2,103,90,128.0,152
3,76,123,119.0,158
4,62,133,110.0,126
5,69,103,NaN,138


Hypothesis:

H0: Wood fibre concentration does not influence the tensile strength of the paper

H1: Wood fibre concentration influences the tensile strength of the paper

## A-priori ANOVA

Calculating averages and summed squares

In [3]:
MEAN=data.mean() #mean of separate classes
display(pd.DataFrame(MEAN,columns=['Yi_']))
Y_=MEAN.mean() #overall mean

#total sum of squares
SST=0
npsT=0
for i in range(data.shape[0]):
    SSTi=0
    for j in range(data.shape[1]):
        if pd.isna(data.iloc[i,j]):
            #print('value in data.iloc['+str(i)+','+str(j)+'] is nan')
            continue
        SSTi=SSTi+(data.iloc[i,j]-Y_)**2
        npsT=npsT+1
    SST=SST+SSTi
npsT=npsT-1
print('Value of SST is {:.3f}'.format(SST))
print('Number of degrees of freedom T is {:.0f}'.format(npsT))

#sum of squares between classes
SSA=0
npsA=0
for i in range(data.shape[0]):
    SSAi=0
    for j in range(data.shape[1]):
        if pd.isna(data.iloc[i,j]):
            #print('value in data.iloc['+str(i)+','+str(j)+'] is nan')
            continue
        SSAi=SSAi+(MEAN[j]-Y_)**2
    SSA=SSA+SSAi
npsA=data.shape[1]-1
print('Value of SSA is {:.3f}'.format(SSA))
print('Number of degrees of freedom A is {:.0f}'.format(npsA))

#sum of squares within classes
SSE=0
npsE=0
for i in range(data.shape[0]):
    SSEi=0
    for j in range(data.shape[1]):
        if pd.isna(data.iloc[i,j]):
            #print('value in data.iloc['+str(i)+','+str(j)+'] is nan')
            continue
        SSEi=SSEi+(data.iloc[i,j]-MEAN[j])**2
        npsE=npsE+1
    SSE=SSE+SSEi
npsE=npsE-data.shape[1] 
print('Value of SSE is {:.3f}'.format(SSE))
print('Number of degrees of freedom E is {:.0f}'.format(npsE))

#test
print(SST==SSA+SSE)
print(npsT==npsA+npsE)

,Yi_
5%,68.833333
10%,108.166667
15%,115.600000
20%,146.166667


Value of SST is 24141.170
Number of degrees of freedom T is 22
Value of SSA is 18187.470
Number of degrees of freedom A is 3
Value of SSE is 5953.700
Number of degrees of freedom E is 19
True
True


Calculating statistic F

In [4]:
#sample variance
MSA=SSA/npsA
MSE=SSE/npsE
#statisctic F
F=MSA/MSE
#nu1,nu2 (parameters of Fischer's distribution)
nu1=npsA
nu2=npsE

print('Value of statistic F is {:.3f}'.format(F))

Fcrit=f.ppf(1-alpha,nu1,nu2)
print('Value of Fcrit is {:.3f}'.format(Fcrit))

#actual risk for the rejection of the H0
p_val=1-f.cdf(F,nu1,nu2)
print('Actual risk for wrong decision if we reject the H0 is {}'.format(p_val))

#ANOVA by built-in function
print(f_oneway(data.iloc[:6,0],data.iloc[:6,1],data.iloc[:5,2],data.iloc[:6,3]))

Value of statistic F is 19.347
Value of Fcrit is 3.127
Actual risk for wrong decision if we reject the H0 is 5.336871767513962e-06
F_onewayResult(statistic=19.34556613558783, pvalue=5.34001205240135e-06)


A small difference between statistic F, calculated manually and with built-in function can be observed. The same analysis (for the same data) was performed in the Excel manually and by built-in function and the same difference was observed. This difference, if sizes of the classes are the same, however disappears.

## Posteriori ANOVA

In order to obtain which class is statistically different than other classes, we have to perform the posteriori ANOVA. The Bonferonni method that was used here, runs overall 6 t-tests for differences in means of all pairs so that the probability of making one or more type 1 errors over the 6 tests is not more than 5%.

In [5]:
nComp=binom(data.shape[1],2)
alphaBonf=alpha/nComp

print('Number of t-tests to be performed is {}.'.format(int(nComp)))
print('Value of alpha for the individual t-tests according to the Bonferonni is {:.6f}.'.format(alphaBonf))

combList=list(combinations(np.arange(0,data.shape[1],1), 2)) #list of combinations to be compared
print('Pairs for the t-test are {}'.format(combList))

pValList=[]
signDiffPairsList=[]
for i in range(data.shape[0]):
    pValList.append(ttest_ind(data.iloc[:,combList[i][0]],data.iloc[:,combList[i][1]],equal_var=False,nan_policy='omit')[1])
    if pValList[i]<=alphaBonf:
        signDiffPairsList.append(combList[i])
            
print('pValues for individual pairs are {}'.format(pValList))
print('Significantly different pairs according to Bonferonni are {}'.format(signDiffPairsList))

BonfTable=np.zeros((data.shape[1]+2,data.shape[1]+2))
BonfTable[2:,0]=sorted(MEAN,reverse=True)
BonfTable[0,2:]=sorted(MEAN,reverse=True)
BonfTable[2:,1]=np.argsort(-MEAN)
BonfTable[1,2:]=np.argsort(-MEAN)

for i in range(2,BonfTable.shape[0]):
    for j in range(2,BonfTable.shape[1]):
        for k in range(len(signDiffPairsList)):
            if signDiffPairsList[k]==(BonfTable[i,1],BonfTable[1,j]):
                BonfTable[i,j]=1
            elif signDiffPairsList[k]==(BonfTable[1,j],BonfTable[i,1]):
                BonfTable[i,j]=1
                
display(pd.DataFrame(BonfTable[1:,1:].astype(int)))

Number of t-tests to be performed is 6.
Value of alpha for the individual t-tests according to the Bonferonni is 0.008333.
Pairs for the t-test are [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
pValues for individual pairs are [0.00575681541814036, 0.001086857975201461, 2.9931751105389414e-05, 0.4643492887698597, 0.0054139099705326045, 0.00846888844119131]
Significantly different pairs according to Bonferonni are [(0, 1), (0, 2), (0, 3), (1, 3)]


,0,1,2,3,4
0,0,3,2,1,0
1,3,0,0,1,1
2,2,0,0,0,1
3,1,1,0,0,1
4,0,1,1,1,0


It can be seen from the matrix above that class 2 (15%) is in two groups; it is common with class 3 (20%) and with class 1(10%).